In [ ]:
#This is the dataset for recording DYNAMIC GESTURES
import cv2
import mediapipe as mp
import numpy as np
import os
from collections import deque
import time
import sys
import argparse

# ── DETECT JUPYTER ENVIRONMENT ─────────
def is_jupyter():
    try:
        get_ipython()
        return True
    except NameError:
        return False

# ── CONFIG FOR JUPYTER ────────────────
# Set these manually if running in Jupyter Notebook
GESTURE_NAME = "sos"  # Change this for each gesture
MAX_RECORDINGS = 5  # Number of gesture performances
NUM_VARIATIONS = 10  # Total files per performance (1 original + 9 augmented)

# ── PARSE ARGUMENTS FOR COMMAND LINE ───
if not is_jupyter():
    parser = argparse.ArgumentParser(description="Record gesture sequences for dataset creation.")
    parser.add_argument("--gesture", type=str, default="dislike", help="Name of the gesture to record (e.g., swipe_right, like).")
    parser.add_argument("--samples", type=int, default=5, help="Number of gesture performances to record.")
    parser.add_argument("--variations", type=int, default=10, help="Number of files to save per performance (1 original + augmented).")
    args = parser.parse_args()
    GESTURE_NAME = args.gesture
    MAX_RECORDINGS = args.samples
    NUM_VARIATIONS = args.variations

# ── CONFIG ─────────────────────────────
SEQUENCE_LENGTH = 60
SAVE_DIR = f"dataset/{GESTURE_NAME}"
os.makedirs(SAVE_DIR, exist_ok=True)
MIN_VALID_FRAMES = 50
MOVEMENT_THRESHOLD = 0.1
AUGMENTATION_NOISE = 0.02

# ── INIT ───────────────────────────────
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                       min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
buffer = deque(maxlen=SEQUENCE_LENGTH)
recording = False
sample_count = 0
valid_frames = 0

# FPS counter
start_time = time.time()
frame_count = 0

def augment_sequence(sequence, noise_level=AUGMENTATION_NOISE):
    """Generate an augmented version of the sequence by adding noise."""
    augmented = sequence.copy()
    noise = np.random.uniform(-noise_level, noise_level, sequence.shape)
    augmented += noise
    augmented[:, ::3] = np.clip(augmented[:, ::3], 0, 1)  # x
    augmented[:, 1::3] = np.clip(augmented[:, 1::3], 0, 1)  # y
    return augmented

def check_movement(sequence, threshold=MOVEMENT_THRESHOLD):
    """Check if the sequence has significant movement (for dynamic gestures)."""
    sequence = np.array(sequence)
    x_coords = sequence[:, ::3]
    x_range = np.ptp(x_coords, axis=0).max()
    y_coords = sequence[:, 1::3]
    y_range = np.ptp(y_coords, axis=0).max()
    return max(x_range, y_range) >= threshold

print(f"📸 Recording {MAX_RECORDINGS} performances for gesture: {GESTURE_NAME}")
print(f"Each performance will save {NUM_VARIATIONS} files (1 original + {NUM_VARIATIONS-1} augmented).")
print("📸 Press 's' to start recording a gesture.")
print("❌ Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if time.time() - start_time >= 1.0:
        print(f"FPS: {frame_count}")
        frame_count = 0
        start_time = time.time()

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    landmark_row = None
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmark_row = []
            for lm in hand_landmarks.landmark:
                landmark_row.extend([lm.x, lm.y, lm.z])
            break

    # ── RECORDING GESTURE ───────────
    if recording:
        if landmark_row:
            buffer.append(landmark_row)
            valid_frames += 1
            cv2.putText(frame, f"Recording ({len(buffer)}/{SEQUENCE_LENGTH} frames)",
                        (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            buffer.append([0] * 63)
            cv2.putText(frame, "No hand detected!",
                        (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if len(buffer) >= SEQUENCE_LENGTH:
            sequence = np.array(buffer)
            if sequence.shape == (SEQUENCE_LENGTH, 63):
                if valid_frames >= MIN_VALID_FRAMES:
                    is_static_gesture = GESTURE_NAME in ["like", "dislike"]
                    if is_static_gesture or check_movement(sequence):
                        # Save original sequence
                        np.savetxt(f"{SAVE_DIR}/{sample_count}_original.csv", sequence, delimiter=",")
                        print(f"✅ Saved original sequence: {SAVE_DIR}/{sample_count}_original.csv")
                        # Save augmented sequences
                        for i in range(1, NUM_VARIATIONS):
                            augmented_sequence = augment_sequence(sequence)
                            np.savetxt(f"{SAVE_DIR}/{sample_count}_{i}.csv", augmented_sequence, delimiter=",")
                            print(f"✅ Saved augmented sequence: {SAVE_DIR}/{sample_count}_{i}.csv")
                        sample_count += 1
                    else:
                        print(f"⚠️ Not enough movement (max range < {MOVEMENT_THRESHOLD}). Try again.")
                else:
                    print(f"⚠️ Not enough valid frames ({valid_frames}/{MIN_VALID_FRAMES}). Try again.")
            buffer.clear()
            valid_frames = 0
            recording = False
            if sample_count >= MAX_RECORDINGS:
                print("🟡 All recordings done!")
                break
            else:
                print("🟡 Stopped recording. Press 's' for next sequence.")
    else:
        status = "Idle - Press 's' to record"
        cv2.putText(frame, status, (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 100, 255), 2)

    cv2.imshow(f"Gesture Recorder: {GESTURE_NAME}", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s') and not recording and sample_count < MAX_RECORDINGS:
        recording = True
        buffer.clear()
        valid_frames = 0
        print("🔴 Started recording... Perform the gesture now.")
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

In [3]:
#This is the dataset for recording STATIC GESTURES
import cv2
import mediapipe as mp
import numpy as np
import os
from collections import deque
import time
import sys
import argparse

# ── DETECT JUPYTER ENVIRONMENT ─────────
def is_jupyter():
    try:
        get_ipython()
        return True
    except NameError:
        return False

# ── CONFIG FOR JUPYTER ────────────────
GESTURE_NAME = "peace"  # Change this for each gesture
MAX_RECORDINGS = 5  # Number of gesture performances
NUM_VARIATIONS = 10  # Total files per performance (1 original + 9 augmented)

# ── PARSE ARGUMENTS FOR COMMAND LINE ───
if not is_jupyter():
    parser = argparse.ArgumentParser(description="Record gesture sequences for dataset creation.")
    parser.add_argument("--gesture", type=str, default="like", help="Name of the gesture to record (e.g., swipe_right, like).")
    parser.add_argument("--samples", type=int, default=5, help="Number of gesture performances to record.")
    parser.add_argument("--variations", type=int, default=10, help="Number of files to save per performance (1 original + augmented).")
    args = parser.parse_args()
    GESTURE_NAME = args.gesture
    MAX_RECORDINGS = args.samples
    NUM_VARIATIONS = args.variations

# ── CONFIG ─────────────────────────────
SEQUENCE_LENGTH = 60
SAVE_DIR = f"dataset/{GESTURE_NAME}"
os.makedirs(SAVE_DIR, exist_ok=True)
MIN_VALID_FRAMES = 50
MOVEMENT_THRESHOLD = 0.1
AUGMENTATION_NOISE = 0.01  # Reduced noise for static gestures

# ── INIT ───────────────────────────────
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                       min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
buffer = deque(maxlen=SEQUENCE_LENGTH)
recording = False
sample_count = 0
valid_frames = 0

# FPS counter
start_time = time.time()
frame_count = 0

def augment_sequence(sequence, noise_level=AUGMENTATION_NOISE):
    """Generate an augmented version of the sequence by adding noise."""
    augmented = sequence.copy()
    noise = np.random.uniform(-noise_level, noise_level, sequence.shape)
    augmented += noise
    augmented[:, ::3] = np.clip(augmented[:, ::3], 0, 1)  # x
    augmented[:, 1::3] = np.clip(augmented[:, 1::3], 0, 1)  # y
    return augmented

def check_movement(sequence, threshold=MOVEMENT_THRESHOLD):
    """Check if the sequence has significant movement (for dynamic gestures)."""
    sequence = np.array(sequence)
    x_coords = sequence[:, ::3]
    x_range = np.ptp(x_coords, axis=0).max()
    y_coords = sequence[:, 1::3]
    y_range = np.ptp(y_coords, axis=0).max()
    return max(x_range, y_range) >= threshold

print(f"📸 Recording {MAX_RECORDINGS} performances for gesture: {GESTURE_NAME}")
print(f"Each performance will save {NUM_VARIATIONS} files (1 original + {NUM_VARIATIONS-1} augmented).")
print("📸 Press 's' to start recording a gesture.")
print("❌ Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if time.time() - start_time >= 1.0:
        print(f"FPS: {frame_count}")
        frame_count = 0
        start_time = time.time()

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    landmark_row = None
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmark_row = []
            for lm in hand_landmarks.landmark:
                landmark_row.extend([lm.x, lm.y, lm.z])
            break

    # ── RECORDING GESTURE ───────────
    if recording:
        if landmark_row:
            buffer.append(landmark_row)
            valid_frames += 1
            cv2.putText(frame, f"Recording ({len(buffer)}/{SEQUENCE_LENGTH} frames)",
                        (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            buffer.append([0] * 63)
            cv2.putText(frame, "No hand detected!",
                        (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if len(buffer) >= SEQUENCE_LENGTH:
            sequence = np.array(buffer)
            if sequence.shape == (SEQUENCE_LENGTH, 63):
                if valid_frames >= MIN_VALID_FRAMES:
                    is_static_gesture = GESTURE_NAME in ["like", "dislike"]
                    # Skip movement check for static gestures
                    if is_static_gesture or check_movement(sequence):
                        # Save original sequence
                        np.savetxt(f"{SAVE_DIR}/{sample_count}_original.csv", sequence, delimiter=",")
                        print(f"✅ Saved original sequence: {SAVE_DIR}/{sample_count}_original.csv")
                        # Save augmented sequences
                        for i in range(1, NUM_VARIATIONS):
                            augmented_sequence = augment_sequence(sequence)
                            np.savetxt(f"{SAVE_DIR}/{sample_count}_{i}.csv", augmented_sequence, delimiter=",")
                            print(f"✅ Saved augmented sequence: {SAVE_DIR}/{sample_count}_{i}.csv")
                        sample_count += 1
                    else:
                        print(f"⚠️ Not enough movement (max range < {MOVEMENT_THRESHOLD}). Try again.")
                else:
                    print(f"⚠️ Not enough valid frames ({valid_frames}/{MIN_VALID_FRAMES}). Try again.")
            buffer.clear()
            valid_frames = 0
            recording = False
            if sample_count >= MAX_RECORDINGS:
                print("🟡 All recordings done!")
                break
            else:
                print("🟡 Stopped recording. Press 's' for next sequence.")
    else:
        status = "Idle - Press 's' to record"
        cv2.putText(frame, status, (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 100, 255), 2)

    cv2.imshow(f"Gesture Recorder: {GESTURE_NAME}", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s') and not recording and sample_count < MAX_RECORDINGS:
        recording = True
        buffer.clear()
        valid_frames = 0
        print("🔴 Started recording... Perform the gesture now.")
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

📸 Recording 5 performances for gesture: peace
Each performance will save 10 files (1 original + 9 augmented).
📸 Press 's' to start recording a gesture.
❌ Press 'q' to quit.
FPS: 18
FPS: 30
🔴 Started recording... Perform the gesture now.
FPS: 31
FPS: 30
⚠️ Not enough movement (max range < 0.1). Try again.
🟡 Stopped recording. Press 's' for next sequence.
FPS: 31
